In [ ]:
import sys
sys.path.append("..")
import gb
from gb.sims import rtd
from gb.sims import NSALoss
from gb.sims import cka, gram_linear

In [ ]:
from torch import nn
import numpy as np
from scipy.stats import wasserstein_distance
from itertools import combinations

import matplotlib.pyplot as plt
import torch

## CKA

In [ ]:
nsa_criterion = NSALoss()

In [ ]:
#Set D to be the size of the dataset
D = 2485

In [ ]:
import numpy as np

# Initialize an empty dictionary to store the combined results
combined_dict = {}

# List of file names
path = 'accuracy_vals/'
file_names = ['5.pkl', '10.pkl', '15.pkl', '20.pkl', '25.pkl']
# Iterate through each file
for file_name in file_names:
    # Load the dictionary from the .pkl file
    file_name = path+file_name
    
    data = np.load(file_name, allow_pickle=True)
    
    # Iterate through the keys in the loaded dictionary
    for gnn_architecture, accuracy_data in data.items():
        # Check if the GNN architecture key exists in the combined_dict
        if gnn_architecture not in combined_dict:
            combined_dict[gnn_architecture] = {}
        
        # Iterate through the second-level keys (type of training)
        for training_type, accuracy_values in accuracy_data.items():
            # Check if the training type key exists in the combined_dict
            if training_type not in combined_dict[gnn_architecture]:
                combined_dict[gnn_architecture][training_type] = []
            
            # Append the accuracy values to the combined dictionary
            combined_dict[gnn_architecture][training_type].append(accuracy_values)

In [ ]:
combined_dict

## Generate similarity values

In [ ]:
attack_type = 'gp'
#attack_type = 'ge'

In [ ]:
###Algorithm
#Attack type - fixed, run 2 times, for ge, gp
value_dict = {}
#Iterate through architectures
for arch in ['gcn','gcnsvd','gnnguard','prognn','grand']:
    value_dict[arch]={}
    #Layers
    for layer in ['conv0','conv1']:
        #Iterate through perturbations
        value_dict[arch][layer]={}
        value_dict[arch][layer]['cka']=[]
        value_dict[arch][layer]['rtd']=[]
        value_dict[arch][layer]['nsa']=[]
        for ptb_rate in [5,10,15,20,25]:
            print(arch,layer,ptb_rate)
            #Compare the clean embedding for the architecture
            clean_path = 'feature_vals/'+arch+'_'+'clean_'+str(ptb_rate)+'.npz'
            print(clean_path)
            with np.load(clean_path) as loader1:
                clean_vals = dict(loader1)
            #Against the embedding for the same architecture with attack and ptb rate of choice
            adv_path = 'feature_vals/'+arch+'_'+attack_type+'_'+str(ptb_rate)+'.npz'
            print(adv_path)
            with np.load(adv_path) as loader2:
                attack_vals = dict(loader2)
            X = clean_vals[layer]
            Y = attack_vals[layer]
            #Given X and Y now
            #Calculate CKA
            print("Calculating CKA")
            cka_from_examples = cka(gram_linear(X), gram_linear(Y))
            cka_val = (1-cka_from_examples)

            value_dict[arch][layer]['cka'].append(cka_val)
            #Calculate RTD
            print("Calculating RTD")
            full_list_indices = np.random.choice(range(D),400, replace=False)
            A = X[full_list_indices]
            B = Y[full_list_indices]
            rtd_val=rtd(A,B)
            print(rtd_val)
            value_dict[arch][layer]['rtd'].append(rtd_val)
            #Calculate NSA
            print("Calculating NSA")
            nsa_val=nsa_criterion(torch.tensor(X),torch.tensor(Y))
            value_dict[arch][layer]['nsa'].append(nsa_val.item())

In [ ]:
import matplotlib.pyplot as plt

def plot_line_graph(val_dict,layer,metric):
    archs = val_dict.keys()
    for arch in archs:
        y = val_dict[arch][layer][metric]
        x = range(len(y))
        plt.plot(x, y, label=arch)
    plt.xlabel('Perturbation rate',fontsize=19)
    plt.ylabel(metric.upper()+" values",fontsize=19)
    plt.title(metric.upper()+' value vs Perturbation rate for: '+layer)
    plt.legend()
    x_ticks = [5,10,15,20,25]
    plt.xticks(x, x_ticks)  # Customize x-tick values
    #plt.show()
    savestr=f'g_{metric}_{layer}.jpg'
    plt.savefig(savestr)

plot_line_graph(value_dict,'conv0','cka')
plot_line_graph(value_dict,'conv1','cka')

In [ ]:
import matplotlib.pyplot as plt

def plot_line_graph(val_dict,layer,metric):
    archs = val_dict.keys()
    for arch in archs:
        y = val_dict[arch][layer][metric]
        x = range(len(y))
        plt.plot(x, y, label=arch)
    plt.xlabel('Perturbation rate',fontsize=19)
    plt.ylabel(metric.upper()+" values",fontsize=19)
    plt.title(metric.upper()+' value vs Perturbation rate for: '+layer)
    plt.legend()
    x_ticks = [5,10,15,20,25]
    plt.xticks(x, x_ticks)  # Customize x-tick values
    #plt.show()
    savestr=f'ge_{metric}_{layer}.jpg'
    plt.savefig(savestr)

# Example usage

plot_line_graph(value_dict,'conv0','rtd')
plot_line_graph(value_dict,'conv1','rtd')

In [ ]:
import matplotlib.pyplot as plt

def plot_line_graph(val_dict,layer,metric):
    archs = val_dict.keys()
    for arch in archs:
        y = val_dict[arch][layer][metric]
        x = range(len(y))
        plt.plot(x, y, label=arch)
    plt.xlabel('Perturbation rate',fontsize=19)
    plt.ylabel(metric.upper()+" value",fontsize=19)
    #plt.title(metric.upper()+' value vs Perturbation rate for: '+layer)
    plt.legend()
    x_ticks = [5,10,15,20,25]
    plt.xticks(fontsize=17)  # Adjust the fontsize as needed
    plt.yticks(fontsize=17)  # Adjust the fontsize as needed
    plt.xticks(x, x_ticks)  # Customize x-tick values
#    plt.show()
    savestr=f'gp_{metric}_{layer}.jpg'
    plt.savefig(savestr)

# Example usage

plot_line_graph(value_dict,'conv0','nsa')
plot_line_graph(value_dict,'conv1','nsa')

In [ ]:
import matplotlib.pyplot as plt

# Define the perturbation values (x-axis)
perturbations = list(range(5, 26, 5))

# Define the data lists
GCN = 100-combined_dict['GCN']['evas']
GCN_SVD = 100-combined_dict['GCNSVD']['evas']
GNNGuard = 100-combined_dict['GNNGuard']['evas']
ProGNN = 100-combined_dict['ProGNN']['evas']
GRAND = 100-combined_dict['GRAND']['evas']


# Create the line graphs
plt.plot(perturbations, GCN, label='GCN')
plt.plot(perturbations, GCN_SVD, label='GCN-SVD')
plt.plot(perturbations, GNNGuard, label='GNNGuard')
plt.plot(perturbations, ProGNN, label='ProGNN')
plt.plot(perturbations, GRAND, label='GRAND')

# Add labels and legend
plt.xlabel('Perturbation Rate',fontsize=19)
plt.ylabel('Misclassification Rate',fontsize=19)
#plt.title('Global Evasion')
plt.xticks(perturbations)
plt.xticks(fontsize=17)  # Adjust the fontsize as needed
plt.yticks(fontsize=17)  # Adjust the fontsize as needed
plt.legend()
plt.savefig("ge_misclassification1.jpg")
# Display the plot
#plt.show()

In [ ]:


import matplotlib.pyplot as plt

# Define the perturbation values (x-axis)
perturbations = list(range(5, 26, 5))

# Define the data lists
GCN = 100-combined_dict['GCN']['pois']
GCN_SVD = 100-combined_dict['GCNSVD']['pois']
GNNGuard = 100-combined_dict['GNNGuard']['pois']
ProGNN = 100-combined_dict['ProGNN']['pois']
GRAND = 100-combined_dict['GRAND']['pois']

# Create the line graphs
plt.plot(perturbations, GCN, label='GCN')
plt.plot(perturbations, GCN_SVD, label='GCN-SVD')
plt.plot(perturbations, GNNGuard, label='GNNGuard')
plt.plot(perturbations, ProGNN, label='ProGNN')
plt.plot(perturbations, GRAND, label='GRAND')

# Add labels and legend
plt.xlabel('Perturbation Rate',fontsize=19)
plt.ylabel('Misclassification Rate',fontsize=19)
#plt.title('Global Poisoning')
plt.xticks(perturbations)
plt.xticks(fontsize=17)  # Adjust the fontsize as needed
plt.yticks(fontsize=17)  # Adjust the fontsize as needed
plt.legend()
plt.savefig("gp_misclassification1.jpg")
# Display the plot
#plt.show()